# 0. Setup

In [2]:
import pandas as pd
import requests
import json
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import os
import yfinance as yf

# 1. API Request

In [3]:
url = 'https://api.hkma.gov.hk/public/market-data-and-statistics/daily-monetary-statistics/daily-figures-interbank-liquidity?offset=0&pagesize=999&sortby=end_of_date&sortorder=desc'
response = requests.get(url).text
response = json.loads(response)
df = pd.DataFrame(response['result']['records'])
df['end_of_date'] = pd.to_datetime(df['end_of_date'])

In [4]:
df

,end_of_date,cu_weakside,cu_strongside,disc_win_base_rate,hibor_overnight,hibor_fixing_1m,twi,opening_balance,closing_balance,market_activities,...,forex_trans_t4,other_market_activities_t4,reversal_of_discount_window_t4,interest_payment_issuance_efbn_t4,forecast_aggregate_bal_t4,forex_trans_u,other_market_activities_u,reversal_of_discount_window_u,interest_payment_issuance_efbn_u,forecast_aggregate_bal_u
0,2025-08-19,7.85,7.75,4.75,2.63,2.57417,103.7,53716,53716,+0,...,None,None,None,None,NaN,+0,+0,-0,-566,53311
1,2025-08-18,7.85,7.75,4.75,1.88,2.01393,103.4,53716,53716,+0,...,None,None,None,None,NaN,+0,+0,-0,-355,53311
2,2025-08-15,7.85,7.75,4.75,0.83,1.45095,103.4,57091,53716,-3376,...,None,None,None,None,NaN,+0,+0,-0,-335,53311
3,2025-08-14,7.85,7.75,4.75,0.32,1.04524,102.9,64156,57091,-7065,...,None,None,None,None,NaN,+0,+0,-0,-404,53311
4,2025-08-13,7.85,7.75,4.75,0.16,0.91036,103.0,64062,64156,+0,...,None,None,None,None,NaN,+0,+0,-0,-404,53311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2021-08-09,7.85,7.75,0.50,0.01,0.07661,101.5,457457,457457,0,...,None,None,None,None,NaN,0,0,0,-1,457454
995,2021-08-06,7.85,7.75,0.50,0.01,0.07768,101.4,457457,457457,0,...,None,None,None,None,NaN,0,0,0,-1,457454
996,2021-08-05,7.85,7.75,0.50,0.01,0.07768,101.3,457457,457457,0,...,None,None,None,None,NaN,0,0,0,-3,457454
997,2021-08-04,7.85,7.75,0.50,0.02,0.08125,101.2,457456,457457,0,...,None,None,None,None,NaN,0,0,0,-3,457454


# 2.0 HIBOR & Liquidity

## 2.1 HIBOR (Overnight & 1-Month)

In [5]:
hibor_df = df[['end_of_date', 'hibor_overnight', 'hibor_fixing_1m']]

In [6]:
hibor_fig = px.line(hibor_df,
             x='end_of_date', y=['hibor_overnight', 'hibor_fixing_1m'],
             title='HIBOR Rates (Overnight & 1-Month)',
             labels={'end_of_date': 'Date', 'value': 'HIBOR Rate (%)'},
             )

last_date = hibor_df['end_of_date'].iloc[0]
last_overnight = hibor_df['hibor_overnight'].iloc[0]
last_1m = hibor_df['hibor_fixing_1m'].iloc[0]

hibor_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>Overnight: {last_overnight}%<br>1M: {last_1m}%",
    xref="paper", yref="paper",
    x=1, y=1.2,  
    showarrow=False,
    bgcolor="rgba(1, 108, 2, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

hibor_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

hibor_fig.show()

## 2.2 Aggregate Balance Open & Close

In [7]:
aggreBal_df = df[['end_of_date', 'opening_balance', 'closing_balance']].head(200)
aggreBal_df['day_change'] = aggreBal_df['closing_balance'] - aggreBal_df['opening_balance']
aggreBal_df['high'] = aggreBal_df[['opening_balance', 'closing_balance']].max(axis=1)
aggreBal_df['low'] = aggreBal_df[['opening_balance', 'closing_balance']].min(axis=1)
aggreBal_df

,end_of_date,opening_balance,closing_balance,day_change,high,low
0,2025-08-19,53716,53716,0,53716,53716
1,2025-08-18,53716,53716,0,53716,53716
2,2025-08-15,57091,53716,-3375,57091,53716
3,2025-08-14,64156,57091,-7065,64156,57091
4,2025-08-13,64062,64156,94,64156,64062
...,...,...,...,...,...,...
195,2024-11-01,44726,44726,0,44726,44726
196,2024-10-31,44726,44726,0,44726,44726
197,2024-10-30,44790,44726,-64,44790,44726
198,2024-10-29,44790,44790,0,44790,44790


In [8]:
aggreBal_fig = go.Figure(data=[go.Candlestick(
    x=aggreBal_df['end_of_date'],
    open=aggreBal_df['opening_balance'],
    high=aggreBal_df['high'],
    low=aggreBal_df['low'],
    close=aggreBal_df['closing_balance'],
    name='Aggregate Balance',
    showlegend=True,
)])

last_date = aggreBal_df['end_of_date'].iloc[0]
last_open = aggreBal_df['opening_balance'].iloc[0]
last_close = aggreBal_df['closing_balance'].iloc[0]
last_day_change = aggreBal_df['day_change'].iloc[0]

aggreBal_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>Open: {last_open}<br>Close: {last_close}<br>Change: {last_day_change}",
    xref="paper", yref="paper",
    x=1, y=1.2,
    showarrow=False,
    bgcolor="rgba(0, 78, 123, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

aggreBal_fig.update_layout(
    xaxis_rangeslider_visible=True,
    title="Hong Kong Aggregate Balance (Candlestick)",
    xaxis_title="Date",
    yaxis_title="Balance (HKD Million)",
)
aggreBal_fig.show()

# 3.0 FX

# 3.1 Convertibility Undertaking (CU)

In [9]:
usdhkd = yf.Ticker("HKD=X")
usdhkd_df = usdhkd.history(period ='max').reset_index()
usdhkd_df['Date'] = pd.to_datetime(usdhkd_df['Date'], format='%Y-%m-%d')
usdhkd_df = usdhkd_df[['Date', 'Close']]
usdhkd_df = usdhkd_df.rename(columns={'Date': 'end_of_date', 'Close': 'usdhkd_close'})
usdhkd_df['end_of_date'] = pd.to_datetime(usdhkd_df['end_of_date'])
usdhkd_df.set_index('end_of_date', inplace=True)
usdhkd_df.index = usdhkd_df.index.tz_localize(None)

In [10]:
cu_df = df[['end_of_date', 'cu_weakside', "cu_strongside"]]
cu_df['end_of_date'] = pd.to_datetime(cu_df['end_of_date'])
cu_df = cu_df.set_index('end_of_date').join(usdhkd_df, on='end_of_date', how='left')

/var/folders/h_/dmgktz0n27g262j_0g_v4qvm0000gn/T/ipykernel_19204/3866773713.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
cu_df = cu_df.reset_index()
cu_df.sort_values(by='end_of_date', ascending=False, inplace=True)
cu_df

,end_of_date,cu_weakside,cu_strongside,usdhkd_close
0,2025-08-19,7.85,7.75,7.79962
1,2025-08-18,7.85,7.75,7.82471
2,2025-08-15,7.85,7.75,7.83460
3,2025-08-14,7.85,7.75,7.84920
4,2025-08-13,7.85,7.75,7.84838
...,...,...,...,...
994,2021-08-09,7.85,7.75,7.78020
995,2021-08-06,7.85,7.75,7.77637
996,2021-08-05,7.85,7.75,7.77555
997,2021-08-04,7.85,7.75,7.77710


In [12]:
cu_fig = px.line(cu_df,
             x='end_of_date', y=['usdhkd_close', 'cu_weakside', 'cu_strongside'],
             title='USD/HKD and Currency Pegs',
             labels={'end_of_date': 'Date', 'value': 'Value (HKD)'},
             )


last_date = cu_df['end_of_date'].iloc[0]
last_usdhkd_close = round(cu_df['usdhkd_close'].iloc[0],4)

cu_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>usdhkd: {last_usdhkd_close}",
    xref="paper", yref="paper",
    x=1, y=1.2, 
    showarrow=False,
    bgcolor="rgba(108, 1, 2, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

cu_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

cu_fig.show()

# 3.2 Trade-Weighted Index, TWI

In [13]:
hkdtwi_df = df[['end_of_date', 'twi']]

In [14]:
hkdtwi_fig = px.line(hkdtwi_df,
             x='end_of_date', y='twi',
             title='HKD Trade-Weighted Index (TWI)',
             labels={'end_of_date': 'Date', 'value': 'Value'},
             )


last_date = hkdtwi_df['end_of_date'].iloc[0]
last_twi = hkdtwi_df['twi'].iloc[0]

hkdtwi_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>TWI(HKD): {last_twi}",
    xref="paper", yref="paper",
    x=1, y=1.2,
    showarrow=False,
    bgcolor="rgba(156, 33, 315, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

hkdtwi_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

hkdtwi_fig.show()

# 4.0 Stock Market

In [15]:
hsi = yf.Ticker("^HSI")
hsi_df = hsi.history(period ='10y').reset_index()
hsi_df.set_index('Date', inplace=True)
hsi_df.index = hsi_df.index.tz_localize(None)
hsi_df.sort_values(by='Date', ascending=False, inplace=True)
hsi_df['day_change'] = hsi_df['Close'] - hsi_df['Open']
hsi_df = hsi_df.reset_index()
hsi_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,day_change
0,2025-08-19,25241.289062,25260.580078,25083.199219,25122.900391,0,0.0,0.0,-118.388672
1,2025-08-18,25293.339844,25466.099609,25176.849609,25176.849609,3720500000,0.0,0.0,-116.490234
2,2025-08-15,25322.099609,25356.539062,25167.810547,25270.070312,4157900000,0.0,0.0,-52.029297
3,2025-08-14,25766.619141,25766.619141,25449.800781,25519.320312,3330000000,0.0,0.0,-247.298828
4,2025-08-13,25176.519531,25613.669922,25176.519531,25613.669922,3271700000,0.0,0.0,437.150391
...,...,...,...,...,...,...,...,...,...
2455,2015-08-25,21119.529297,21871.400391,20865.259766,21404.960938,3588800000,0.0,0.0,285.431641
2456,2015-08-24,21605.970703,21679.449219,21136.480469,21251.570312,3579900000,0.0,0.0,-354.400391
2457,2015-08-21,22343.250000,22492.789062,22185.849609,22409.619141,2490600000,0.0,0.0,66.369141
2458,2015-08-20,22973.869141,23033.970703,22610.529297,22757.470703,2339000000,0.0,0.0,-216.398438


In [16]:
hsi_fig = px.line(hsi_df,
             x='Date', y='Close',
             title='HSI',
             labels={'Date': 'Date', 'value': 'Value'},
             )


last_date = hsi_df['Date'].iloc[0]
last_hsi = round(hsi_df['Close'].iloc[0],2)

hsi_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>Index(HKD): {last_hsi}",
    xref="paper", yref="paper",
    x=1, y=1.2, 
    showarrow=False,
    bgcolor="rgba(156, 33, 315, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

hsi_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

hsi_fig.show()

In [17]:
# hibor_fig.write_html('assets/hibor_fig.html')
# aggreBal_fig.write_html('assets/aggreBal_fig.html')
# cu_fig.write_html('assets/cu_fig.html')
# hkdtwi_fig.write_html('assets/hkdtwi_fig.html')
# hsi_fig.write_html('assets/hsi_fig.html')

In [29]:
figs = [
    (hsi_fig, "HSI"),
    (hibor_fig, "HIBOR"),
    (aggreBal_fig, "AggreBal"),
    (cu_fig, "Currency"),
    (hkdtwi_fig, "HKDTWI"),
]

html = """<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Charts</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body { margin: 20px; text-align: center; }
        div { margin: 30px auto; width: 80%; }
    </style>
</head>
<body>
    <h1>Charts</h1>
"""
for i, (fig, title) in enumerate(figs, 1):
    html += "<hr>\n"
    html += f"<h2>{title}</h2>\n"
    html += fig.to_html(full_html=False, div_id=f"chart{i}", include_plotlyjs=True)

html += "</body>\n</html>"
output_path = os.path.abspath(os.path.join(os.path.abspath(os.path.join(os.getcwd(), "..")), ".."))+ "/docs/charts.html"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(html)